In [8]:
#import gym
#import pennylane
from pyquil import Program
from pyquil.api import get_qc, WavefunctionSimulator, local_qvm
from pyquil.gates import *
import numpy as np
import math
import random as pr

#import os, inspect, sys
#import matplotlib.pyplot as plt
#import sys
#from operator import xor
#sys.path.insert(0, 'tests/')
#sys.path.insert(0, 'auxiliary_functions/')

#from auxiliary_functions.auxiliary_rb import *

%matplotlib inline

In [2]:
qc_name = 'Aspen-4-16Q-A'

qc = get_qc(qc_name, as_qvm = True)

qubits = qc.qubits()

rand = [np.random.choice(2) for i in range(6)]
#n_samples = 1

In [7]:
def circ(state, policy,n_samples):
    p = Program()
    p += [H(i) for i in qubits]
    p += [RZ(state[i], i) for i in qubits]
    for i in range(len(qubits)):
        for j in range(i+1):
            if i != j:
                p += CZ(i,j)
    p += [RZ(state[0]*state[1], i),RZ(state[1]*state[0], i),RZ(state[2]*state[3], i),RZ(state[3]*state[1], i)]
    p += [RZ(policy[i], i) for i in qubits]
    count = 0
    for i in range(len(qubits)):
        for j in range(i+1):
            if i != j:
                if rand[count] == 1:
                    p += CZ(i,j)
                count+=1
    n = len(qubits)
    ro = p.declare('ro', 'BIT',n)
    p += [MEASURE(qubits[i], ro[i]) for i in qubits]
    
    p.wrap_in_numshots_loop(n_samples)
    quil_ex = qc.compile(p)
    samples = qc.run(quil_ex) # Use 'run' not 'run_and_measure' here
    if n_samples == 1:
        votes = [1 if np.count_nonzero(samples[:,i]) == 1 else 0 for i in range(4)]
        #print(votes)
        vote1 = xor(votes[0],votes[1])
        vote2 = xor(votes[2],votes[3])
        #print(samples, votes, vote1, vote2)
        return xor(vote1, vote2)
    else:
        actions = []
        for j in range(n_samples):
            votes = [1 if np.count_nonzero(samples[j,i]) == 1 else 0 for i in range(4)]
            #print(votes)
            vote1 = xor(votes[0],votes[1])
            vote2 = xor(votes[2],votes[3])
            #print(samples, votes, vote1, vote2)
            actions.append( xor(vote1, vote2))
        return actions

In [9]:
#credit to Jeremy Stober for the original simulation library github.com/stober, code adapted from here.


class CartPole(object):

    def __init__(self, x = 0.0, xdot = 0.0, theta = 0.0, thetadot = 0.0):
        self.x = x
        self.xdot = xdot
        self.theta = theta
        self.thetadot = thetadot

        # some constants
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5		  # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02		  # seconds between state updates
        self.fourthirds = 1.3333333333333

    def failure(self):
        twelve_degrees = 0.2094384
        if (not -2.4 < self.x < 2.4) or (not -twelve_degrees < self.theta < twelve_degrees):
            return True
        else:
            return False

    def reset(self):
        self.x,self.xdot,self.theta,self.thetadot = (0.0,0.0,0.0,0.0)


    def single_episode(self, policy):
        self.reset()
        #if policy is None: policy = self.random_policy

        trace = []
        next_action = circ([self.x,self.xdot,self.theta,self.thetadot], policy, 1)
        while not self.failure():
            #state = self.move(next_action)
            #next_action = circ(state, policy)
            #print('nudge',next_action,state)
            #trace.append([1])
            pstate, paction, reward, state = self.move(next_action)
            next_action                    =  circ(state, policy, n_samples)
            trace.append([pstate, paction])
        #gamma = 0.9
        #tot_rew = len(trace)
        return trace #(1-gamma**tot_rew)/(1-gamma)


    def reward(self):
        if self.failure():
            return 0.0
        else:
            return 1.0

    def move(self, action, boxed=False): # binary L/R action
        force = 0.0
        if action > 0:
            force = self.force_mag
        else:
            force = -self.force_mag

        costheta = math.cos(self.theta)
        sintheta = math.sin(self.theta)

        tmp = (force + self.polemass_length * (self.thetadot ** 2) * sintheta) / self.total_mass;
        thetaacc = (self.gravity * sintheta - costheta * tmp) / (self.length * (self.fourthirds - self.masspole * costheta ** 2 / self.total_mass))
        xacc = tmp - self.polemass_length * thetaacc * costheta / self.total_mass

        (px,pxdot,ptheta,pthetadot) = (self.x, self.xdot, self.theta, self.thetadot)
        #pstate = self.state()

        self.x += self.tau * self.xdot
        self.xdot += self.tau * xacc
        self.theta += self.tau * self.thetadot
        self.thetadot += self.tau * thetaacc
        return [px,pxdot,ptheta,pthetadot],action,self.reward(),[self.x,self.xdot, self.theta, self.thetadot]

    
def grad_est(state, action, policy):
    n_samples = 5 #this will need to be much higher
    delta = 1e-4
    sample = circ(state, policy, n_samples)
    #[print(sample)]
    prob_est = sample.count(action)/n_samples
    if prob_est == 0:
        prob_est = 1e-4
    #print(prob_est)
    prob_del = []
    for i in range(len(policy)):
        temp_p = policy[:]
        temp_p[i] += delta
        sample = circ(state, temp_p, n_samples)
        prob_est = sample.count(action)/n_samples
        if prob_est == 0:
            prob_est = 1e-4
        prob_del.append(prob_est)
        #print(prob_del)
    return [(np.log(prob_del[i])-np.log(prob_est))/delta if prob_del[i] ==0.0 or prob_est == 0.0 else ((prob_del[i])-(prob_est))/delta for i in range(len(policy))]
    
"""
if __name__ == '__main__':

    cp = CartPole()

    global ccount
    ccount = 1
    print(cp.single_episode(policy))
"""

"\nif __name__ == '__main__':\n\n    cp = CartPole()\n\n    global ccount\n    ccount = 1\n    print(cp.single_episode(policy))\n"

In [11]:
num_episodes = 50
policy = [np.random.random() for i in range(4)]
cp = CartPole()
learning_rate = 0.02
gamma = 0.9
ks = []
n_samples = 5
for i in range(num_episodes):
    episode = cp.single_episode(policy)
    k = len(episode)
    ks.append(k)
    print(k)
    grads = [grad_est(episode[i][0], episode[i][1], policy) for i in range(k)]
    #print('grads :',grads)
    for j in range(k):
        for l in range(4):
            policy[l] += learning_rate*grads[j][l]*(1-gamma**(i+1))/(1-gamma)
            #print('polices :', policy)

IndexError: list index out of range

In [ ]:
circ(state, policy,n_samples)